In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from torch.nn import BCELoss
import torch.optim as optim

from model import Model

In [2]:
WEIGHT_DECAY = 0.01
LR = 0.1
EPOCHS = 1000

In [3]:
df = pd.read_csv('league_of_legends_data_large.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   win           1000 non-null   int64
 1   kills         1000 non-null   int64
 2   deaths        1000 non-null   int64
 3   assists       1000 non-null   int64
 4   gold_earned   1000 non-null   int64
 5   cs            1000 non-null   int64
 6   wards_placed  1000 non-null   int64
 7   wards_killed  1000 non-null   int64
 8   damage_dealt  1000 non-null   int64
dtypes: int64(9)
memory usage: 70.4 KB


,win,kills,deaths,assists,gold_earned,cs,wards_placed,wards_killed,damage_dealt
0,0,16,6,19,17088,231,11,7,15367
1,1,8,8,5,14865,259,10,2,38332
2,0,0,17,11,15919,169,14,5,24642
3,0,19,11,1,11534,264,14,3,15789
4,0,12,7,6,18926,124,15,7,40268


In [4]:
X = df.drop('win', axis=1)
Y = df['win']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [6]:
x_train_scaled_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
x_test_scaled_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [7]:
model = Model(num_features_entrada=x_train_scaled_tensor.shape[1])
criterion = BCELoss()
optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

In [8]:
for x in range(EPOCHS+1):
    model.train()

    optimizer.zero_grad()

    output = model(x_train_scaled_tensor)

    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if x%100 == 0:
        print(f"Epoch: {x}, Loss: {loss}")

Epoch: 0, Loss: 0.7143958806991577
Epoch: 100, Loss: 0.6854559183120728
Epoch: 200, Loss: 0.685261070728302
Epoch: 300, Loss: 0.6852609515190125
Epoch: 400, Loss: 0.685261070728302
Epoch: 500, Loss: 0.685261070728302
Epoch: 600, Loss: 0.6852610111236572
Epoch: 700, Loss: 0.685261070728302
Epoch: 800, Loss: 0.685261070728302
Epoch: 900, Loss: 0.685261070728302
Epoch: 1000, Loss: 0.685261070728302


In [9]:
model.eval()
with torch.no_grad():
    output_train = model(x_train_scaled_tensor)
    output_test = model(x_test_scaled_tensor)

    for e, x in enumerate(output_test):
        if x >= 0.5:
            output_test[e] = 1
        else:
            output_test[e] = 0

    for e, x in enumerate(output_train):
        if x >= 0.5:
            output_train[e] = 1
        else:
            output_train[e] = 0

    result_test = output_test == y_test_tensor
    result_train = output_train == y_train_tensor
    print(result_test.float().mean())
    print(result_train.float().mean())


tensor(0.5050)
tensor(0.5437)


In [10]:
y_test_numpy = y_test_tensor.numpy()
y_train_numpy = y_train_tensor.numpy()

output_test_numpy = output_test.numpy()
output_train_numpy = output_train.numpy()

In [11]:
print(confusion_matrix(y_test_numpy, output_test_numpy))
print(classification_report(y_test_numpy, output_test_numpy))

[[40 55]
 [44 61]]
              precision    recall  f1-score   support

         0.0       0.48      0.42      0.45        95
         1.0       0.53      0.58      0.55       105

    accuracy                           0.51       200
   macro avg       0.50      0.50      0.50       200
weighted avg       0.50      0.51      0.50       200



In [12]:
torch.save(model.state_dict(), 'model.pth')

In [13]:
new_model = Model(num_features_entrada=x_train_scaled_tensor.shape[1])
new_model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [14]:
new_model.eval()
with torch.no_grad():
    output_train = new_model(x_train_scaled_tensor)
    output_test = new_model(x_test_scaled_tensor)

    for e, x in enumerate(output_test):
        if x >= 0.5:
            output_test[e] = 1
        else:
            output_test[e] = 0

    for e, x in enumerate(output_train):
        if x >= 0.5:
            output_train[e] = 1
        else:
            output_train[e] = 0

    result_test = output_test == y_test_tensor
    result_train = output_train == y_train_tensor
    print(result_test.float().mean())
    print(result_train.float().mean())


y_test_numpy = y_test_tensor.numpy()
y_train_numpy = y_train_tensor.numpy()

output_test_numpy = output_test.numpy()
output_train_numpy = output_train.numpy()

print(confusion_matrix(y_test_numpy, output_test_numpy))
print(classification_report(y_test_numpy, output_test_numpy))

tensor(0.5050)
tensor(0.5437)
[[40 55]
 [44 61]]
              precision    recall  f1-score   support

         0.0       0.48      0.42      0.45        95
         1.0       0.53      0.58      0.55       105

    accuracy                           0.51       200
   macro avg       0.50      0.50      0.50       200
weighted avg       0.50      0.51      0.50       200

